Data Wrangling

a.Introduction

a.1 The objective of this project is to build models to predict the price of car as a function of features that define a car and and to study how the variation of the different features affect the price of the car


a.2 The purpose of this notebook is to conduct data wrangling including transforming raw data to a cleaned state that is more suitable for the next phase - EDA

Table of Contents

[b.1 Get data source](#b.1)

[b.2 Row + Column count](#b.2)

[b.3 Type + Semantics](#b.3)

[b.4 Data Quality Check](#b.4)

[b.5 Categorical and Non-categorical characterization](#b.5)

In [159]:
# Import necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from scipy import stats


<a id="b.1"></a>
## b.1 Get data source


In [160]:
# Get data source
car_data = pd.read_csv('CarPrice_Assignment.csv')
car_data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


<a id="b.2"></a>
## b.2 Row + Column count

In [161]:
# Find out the total number of rows and columns
total_rows, total_columns = car_data.shape

print("Total Rows:", total_rows)
print("Total Columns:", total_columns)

Total Rows: 205
Total Columns: 26


<a id="b.3"></a>
## b.3 Type + Semantics

In [162]:
# Get the data types of each column
print(car_data.dtypes)

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object


DATA DICTIONARY						
						
1	Car_ID			Unique id of each observation (Interger)		
2	Symboling 			Its assigned insurance risk rating, A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.(Categorical) 		
3	carCompany			Name of car company (Categorical)		
4	fueltype			Car fuel type i.e gas or diesel (Categorical)		
5	aspiration			Aspiration used in a car (Categorical)		
6	doornumber			Number of doors in a car (Categorical)		
7	carbody			body of car (Categorical)		
8	drivewheel			type of drive wheel (Categorical)		
9	enginelocation			Location of car engine (Categorical)		
10	wheelbase			Weelbase of car (Numeric)		
11	carlength			Length of car (Numeric)		
12	carwidth			Width of car (Numeric)		
13	carheight			height of car (Numeric)		
14	curbweight			The weight of a car without occupants or baggage. (Numeric)		
15	enginetype			Type of engine. (Categorical)		
16	cylindernumber			cylinder placed in the car (Categorical)		
17	enginesize			Size of car (Numeric)		
18	fuelsystem			Fuel system of car (Categorical)		
19	boreratio			Boreratio of car (Numeric)		
20	stroke			Stroke or volume inside the engine (Numeric)		
21	compressionratio			compression ratio of car (Numeric)		
22	horsepower			Horsepower (Numeric)		
23	peakrpm			car peak rpm (Numeric)		
24	citympg			Mileage in city (Numeric)		
25	highwaympg			Mileage on highway (Numeric)		
26	price(Dependent variable)			Price of car (Numeric)		
					

<a id="b.4"></a>
## b.4 Data Quality Check

In [163]:
# Checking for duplicate rows in the dataset
duplicate_rows = car_data[car_data.duplicated()]

# Checking for missing values in each column
missing_values = car_data.isnull().sum()

# Checking for unique values in each column to identify potential categorical columns with limited unique entries
unique_values = car_data.nunique()

# Summarize the data quality check results
data_quality_check = {
    'Duplicate Rows': len(duplicate_rows),
    'Missing Values Per Column': missing_values,
    'Unique Values Per Column': unique_values
}

data_quality_check


{'Duplicate Rows': 0,
 'Missing Values Per Column': car_ID              0
 symboling           0
 CarName             0
 fueltype            0
 aspiration          0
 doornumber          0
 carbody             0
 drivewheel          0
 enginelocation      0
 wheelbase           0
 carlength           0
 carwidth            0
 carheight           0
 curbweight          0
 enginetype          0
 cylindernumber      0
 enginesize          0
 fuelsystem          0
 boreratio           0
 stroke              0
 compressionratio    0
 horsepower          0
 peakrpm             0
 citympg             0
 highwaympg          0
 price               0
 dtype: int64,
 'Unique Values Per Column': car_ID              205
 symboling             6
 CarName             147
 fueltype              2
 aspiration            2
 doornumber            2
 carbody               5
 drivewheel            3
 enginelocation        2
 wheelbase            53
 carlength            75
 carwidth             44
 carheig

It is beneficial to separate the company name and use it as a feature, while discarding the specific model name as it is too specific and could lead to overfitting.
Let's separate the company name from CarName and inspect the unique values to ensure the company names are consistently represented. After that, we can perform one-hot encoding on the categorical variables.

In [164]:
# Separate company name from CarName
car_data['CompanyName'] = car_data['CarName'].apply(lambda x: x.split()[0])

# Check unique values
car_data['CompanyName'].unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

Looking at the unique company names, we see some inconsistencies and possible misspellings: maxda and mazda likely refer to the same company. Nissan and nissan are the same but are treated as different due to capitalization. porsche and porcshce likely refer to the same company. toyota and toyouta likely refer to the same company. vokswagen, volkswagen, and vw likely refer to the same company.

In [165]:
# Correct the company names
car_data['CompanyName'] = car_data['CompanyName'].replace({'maxda': 'mazda', 
                                                           'Nissan': 'nissan', 
                                                           'porcshce': 'porsche', 
                                                           'toyouta': 'toyota', 
                                                           'vokswagen': 'volkswagen', 
                                                           'vw': 'volkswagen'})

# Check the corrected unique values
car_data['CompanyName'].unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [166]:
#To drop the CarName column

car_data=car_data.drop('CarName', axis=1)

The company names have been corrected and are now consistently represented. 

<a id="b.5"></a>
## b.5 Categorical and Non-categorical characterization

In [167]:
# Displaying data types of all columns
column_data_types = car_data.dtypes

# Analyze each column to categorize them
categorical_columns = []
numerical_columns = []

for column, dtype in column_data_types.items():
    if dtype == 'object':
        categorical_columns.append(column)
    else:
        numerical_columns.append(column)

# Print the results
print("Categorical Columns:")
print(categorical_columns)
print("\nNumerical Columns:")
print(numerical_columns)

Categorical Columns:
['fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem', 'CompanyName']

Numerical Columns:
['car_ID', 'symboling', 'wheelbase', 'carlength', 'carwidth', 'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']


Summary

The data wrangling section loads the raw data, checks for missing data, separates company name from car name, then characterizes the category and non-category data. 